## XESMF with H5NetCDF and earthaccess

Requires the upcoming ESMF 8.7 release - https://github.com/pangeo-data/xESMF/issues/380

In [ ]:
import argparse
import itertools

import earthaccess
import numpy as np
import pyproj
import rasterio.transform
import xarray as xr
import xesmf as xe
from common import earthaccess_args

In [ ]:
def make_grid_ds() -> xr.Dataset:
    """
    Modified from ndpyramid - https://github.com/carbonplan/ndpyramid
    """
    dstSRS = "EPSG:3857"
    width = height = 256
    te = [
        -20037508.342789244,
        -20037508.342789244,
        20037508.342789244,
        20037508.342789244,
    ]

    transform = rasterio.transform.Affine.translation(
        te[0], te[3]
    ) * rasterio.transform.Affine.scale((te[2] * 2) / width, (te[1] * 2) / height)

    p = pyproj.Proj(dstSRS)

    grid_shape = (height, width)
    bounds_shape = (height + 1, width + 1)

    xs = np.empty(grid_shape)
    ys = np.empty(grid_shape)
    lat = np.empty(grid_shape)
    lon = np.empty(grid_shape)
    lat_b = np.zeros(bounds_shape)
    lon_b = np.zeros(bounds_shape)

    # calc grid cell center coordinates
    ii, jj = np.meshgrid(np.arange(height) + 0.5, np.arange(width) + 0.5)
    for i, j in itertools.product(range(grid_shape[0]), range(grid_shape[1])):
        locs = [ii[i, j], jj[i, j]]
        xs[i, j], ys[i, j] = transform * locs
        lon[i, j], lat[i, j] = p(xs[i, j], ys[i, j], inverse=True)

    # calc grid cell bounds
    iib, jjb = np.meshgrid(np.arange(height + 1), np.arange(width + 1))
    for i, j in itertools.product(range(bounds_shape[0]), range(bounds_shape[1])):
        locs = [iib[i, j], jjb[i, j]]
        x, y = transform * locs
        lon_b[i, j], lat_b[i, j] = p(x, y, inverse=True)

    return xr.Dataset(
        {
            "x": xr.DataArray(xs[0, :], dims=["x"]),
            "y": xr.DataArray(ys[:, 0], dims=["y"]),
            "lat": xr.DataArray(lat, dims=["y", "x"]),
            "lon": xr.DataArray(lon, dims=["y", "x"]),
            "lat_b": xr.DataArray(lat_b, dims=["y_b", "x_b"]),
            "lon_b": xr.DataArray(lon_b, dims=["y_b", "x_b"]),
        },
    )

In [ ]:
def regrid(dataset):
    args = earthaccess_args[dataset]
    input_uri = f'{args["folder"]}/{args["filename"]}'
    src = f's3://{args["bucket"]}/{input_uri}'
    target_grid = make_grid_ds()
    fs = earthaccess.get_s3fs_session(daac=args["daac"])
    fsspec_caching = {
        "cache_type": "none",
    }
    with fs.open(src, **fsspec_caching) as f:
        da = xr.open_dataset(f, engine="h5netcdf", mask_and_scale=True)[
            args["variable"]
        ]
        regridder = xe.Regridder(
            da,
            target_grid,
            "nearest_s2d",
            periodic=True,
            extrap_method="nearest_s2d",
            ignore_degenerate=True,
        )
        return regridder(da)

In [ ]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Set environment for the script.")
    parser.add_argument(
        "--dataset",
        default="mursst",
        help="Dataset to resample.",
        choices=["gpm_imerg", "mursst"],
    )
    user_args = parser.parse_args()
    da = regrid(user_args.dataset)